In [ ]:
# load dotenv
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import textwrap
import time

In [ ]:
model_name = "HuggingFaceTB/SmolLM-135M"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
device_map = {"": 0}

model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device_map)
quantized_model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, device_map=device_map
)
dequantized_model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, device_map=device_map
).dequantize()
print(model)
print(quantized_model)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=200):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.1,
        early_stopping=False, #Can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id,
    )
    return outputs

In [ ]:
# Example Input text
input_text = "Tell a short history of humanity with happy ending."
# Example Output function
def example_output_tokens(model, tokenizer, input_text):
    input_sentences = tokenizer(input_text, return_tensors="pt").to('cuda')
    foundational_outputs_sentence = get_outputs(model, input_sentences, max_new_tokens=100)
    return foundational_outputs_sentence

def example_output_text(tokenizer, tokens):
    return tokenizer.batch_decode(tokens, skip_special_tokens=True)

start = time.time()
tokens = example_output_tokens(model, tokenizer, input_text)
print(f"Time taken to generate tokens: {time.time() - start}")
text = example_output_text(tokenizer, tokens)
print(tokens)

In [ ]:
def beautify_text(text):
    print("Generated Output:\n")
    for i, sentence in enumerate(text, 1):
        wrapped_sentence = textwrap.fill(sentence, width=80)
        print(f"Output {i}:\n{wrapped_sentence}\n")

print(beautify_text(text))

In [ ]:
compare_embeddings = model.model.embed_tokens.weight == quantized_model.model.embed_tokens.weight
# check if tensor_1 is all bool values
torch.all(compare_embeddings)

In [ ]:
def get_model_memory_usage(model):
  """Calculates the memory usage of a PyTorch model."""
  total_memory = 0
  for param in model.parameters():
    total_memory += param.element_size() * param.numel()
  return total_memory

# Calculate memory usage for the quantized model
quantized_model_memory = get_model_memory_usage(quantized_model)
print(f"Quantized model memory usage: {quantized_model_memory / (1024**2):.2f} MB")

# Calculate memory usage for the original model
original_model_memory = get_model_memory_usage(model)
print(f"Original model memory usage: {original_model_memory / (1024**2):.2f} MB")

In [ ]:
# Count the number of attention heads in the model
num_attention_heads = model.config.num_attention_heads
print(f"Number of attention heads: {num_attention_heads}")

In [ ]:
print(model.model.layers[0].self_attn.q_proj.weight)

In [ ]:
# set all tensors of model to not require gradients
for param in model.parameters():
    param.requires_grad = False

In [ ]:
print(model)
print(quantized_model)

In [ ]:
# replace embedding
# model.model.embed_tokens.weight.data = dequantized_model.model.embed_tokens.weight.data
model.model.embed_tokens.weight.data = quantized_model.model.embed_tokens.weight.data
# replace first 30 att heads from quantized model to original model
for i in range(2):
    model.model.layers[i].self_attn.q_proj = quantized_model.model.layers[i].self_attn.q_proj
    model.model.layers[i].self_attn.k_proj = quantized_model.model.layers[i].self_attn.k_proj
    model.model.layers[i].self_attn.v_proj = quantized_model.model.layers[i].self_attn.v_proj

print("Hybrid model memory usage: ", get_model_memory_usage(model) / (1024**2))
# run with torch autocast
with torch.amp.autocast(device_type="cuda"):
    start = time.time()
    output_tokens = example_output_tokens(model, tokenizer, input_text)
    print("Time taken for hybrid model: ", time.time() - start)
output_text = example_output_text(tokenizer, output_tokens)



In [ ]:
for att_head in model.model.layers:
    print(att_head.self_attn.q_proj)
    print(att_head.self_attn.k_proj)
    print(att_head.self_attn.v_proj)

In [ ]:
print(beautify_text(output_text))